# Parse Input

In [7]:
with open("day16_input.txt") as f:
    puzzle_input = f.read().split("\n\n")

In [35]:
def parse_tickets(ticket_list):
    tickets = ticket_list.split("\n")[1:]
    tickets = [ticket.split(",") for ticket in tickets]
    tickets = [[int(x) for x in ticket] for ticket in tickets]
    return tickets

In [41]:
valid_fields = puzzle_input[0].split("\n")
my_ticket = parse_tickets(puzzle_input[1])[0]
other_tickets = parse_tickets(puzzle_input[2])

In [71]:
def clean_fields(fields):
    fields_dict = {}
    
    for field in fields:
        field_name, vals = field.split(": ")
        ranges = [[int(x) for x in rng.split("-")] for rng in vals.split(" or ")]

        fields_dict[field_name] = ranges
    return fields_dict

In [74]:
valid_fields = clean_fields(valid_fields)

# Part 1

In [101]:
def get_overall_ranges(valid_field_values):
    
    rng1_min = min([rng1[0] for rng1,rng2 in valid_field_values])
    rng1_max = max([rng1[1] for rng1,rng2 in valid_field_values])
    
    rng2_min = min([rng2[0] for rng1,rng2 in valid_field_values])
    rng2_max = max([rng2[1] for rng1,rng2 in valid_field_values])

    # If they overlap - I could have hard coded this but I didn't
    if rng2_min < rng1_max:
        return [(rng1_min, rng2_max)]
    else:
        return [(rng1_min, rng1_max), (rng2_min, rng2_max)]

In [103]:
overall_valid_range = get_overall_ranges(valid_fields.values())

In [136]:
def check_valid_num(num, rng):
    lower,upper = rng[0]
    # we know there is only one range - this I did hard code :(
    return (num >= lower) & (num <= upper)

In [133]:
def check_valid_tickets(tickets, rng):
    
    invalid_nums = []
    for ticket in tickets:
        invalid_nums.extend([x for x in ticket if not check_valid_num(x,rng)])
    return sum(invalid_nums)

In [135]:
check_valid_tickets(other_tickets, overall_valid_range)

27898

# Part 2